In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use('ggplot') # optional: for ggplot-like style
import folium
from scipy import stats  #for p value
  
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Reading the CSV file 
file_name_2016 = "/kaggle/input/sea-building-energy-benchmarking/2016-building-energy-benchmarking.csv"
file_name_2015 = "/kaggle/input/sea-building-energy-benchmarking/2015-building-energy-benchmarking.csv"
df = pd.read_csv(file_name_2016)
df1 = pd.read_csv(file_name_2015)
display(df.head())  #allows to print the dataframes 

#pd.set_option('max_columns', 40)        # set the max columns to be displayed
#pd.set_option('display.max_colwidth',20)
#pd.reset_option('all')         # resets every parameters
print(pd.options.display.max_colwidth)
print(pd.options.display.max_columns)
print(pd.options.display.max_rows)
df1.head()

In [ ]:
print("Shape of the DataFrame 2016:",df.shape,"\n")
print("Shape of the DataFrame 2015:",df1.shape,"\n")
#df.columns


In [ ]:
df.info()
#df1.info()
#df1.count()


In [ ]:
# Do this while building model, as of now go with replacing missing values
# df = df.dropna(subset=['ENERGYSTARScore'])
# df.info()



## Some information about various variables
1. PropertyGFATotal : Property Gross floor area
2. NumberofBuildings : No. of building in an allocated ID, (eg: hotel, big complex)
3. SiteEUI(kBtu/sf) : Energy Use Index (EUI) represents kBtu per square foot of conditioned floor area. ( British Thermal Unit(Btu) are used in building energy use tracking and heating system sizing)
4. ENERGYSTARScore: Ranges from 0-100(bad-good),  building’s energy consumption measures up against similar buildings nationwide. Score of 75+ : building is a top performer — and may be eligible for ENERGY STAR certification
5. ListOfAllPropertyUseTypes: (Purpose of using property, eg: hotel, parking,etc)        
6. PrimaryPropertyType: Primary purpose of use
7. SiteEUIWN(kBtu/sf) : ? 
8. Site Vs Source : ?
9. COmpliance Status : ?

<p> &nbsp; </p>
<!-- This is commented -->  


In [ ]:
# counts the number of times each data has appeared in a column

#print(df['PropertyGFAParking'].value_counts())
#df[['PropertyGFAParking','PropertyGFATotal']]
#print(df['BuildingType'].value_counts())
df['ListOfAllPropertyUseTypes'].value_counts()


In [ ]:
viz = df[['YearBuilt', 'SiteEUI(kBtu/sf)']]
viz.hist()
df.hist(column='YearBuilt', bins=500)
plt.show()

In [ ]:

df.describe()


In [ ]:
df[["ENERGYSTARScore"]]

In [ ]:
# correlation matrix

fig, ax = plt.subplots(figsize=(20,20))         # Sample figsize in inches
sns.heatmap(df.corr(), annot=True , ax = ax , cmap = 'RdYlGn', center=0)     # annot : shows value too

#df.corr()


In [ ]:
display(sns.regplot(x="SiteEnergyUseWN(kBtu)", y="TotalGHGEmissions", data=df))
sns.jointplot(df.loc[:,'SiteEnergyUseWN(kBtu)'], df.loc[:,'TotalGHGEmissions'], kind="regg", color="#ce1414")

# Data Wrangling



In [ ]:
df.info()

In [ ]:
#Few analysis in order to deal efficiently with missing value
    
display(df[[ 'Electricity(kWh)' , 'Electricity(kBtu)']].corr())
df[['NaturalGas(therms)', 'NaturalGas(kBtu)']].corr()
# hence keep only one

In [ ]:
# remove the unnecessary columns 

df.drop(['Electricity(kWh)','NaturalGas(therms)', 'TaxParcelIdentificationNumber',"ListOfAllPropertyUseTypes", "SecondLargestPropertyUseType" ,"SecondLargestPropertyUseTypeGFA" , "ThirdLargestPropertyUseType" , 
        "ThirdLargestPropertyUseTypeGFA" , "ZipCode","Comments", "Outlier" ,"YearsENERGYSTARCertified" , "DefaultData"], axis = 1, inplace=True)
df.head()



In [ ]:
# converting the data type
# eg. year from int to obj
#df[['LargestPropertyUseType']].value_counts()
df.info()

In [ ]:
# replaces the missing values with mean

df['NumberofBuildings'].value_counts()

replace_mean_list = ['ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kBtu)',
       'NaturalGas(kBtu)', 'TotalGHGEmissions',
       'GHGEmissionsIntensity' ,'LargestPropertyUseTypeGFA' ]

for i in replace_mean_list:
    df[i].fillna(df[i].mean(), inplace=True)

df['NumberofBuildings'].fillna(df['NumberofBuildings'].median(), inplace=True)
df['LargestPropertyUseType'].fillna('Multifamily Housing',inplace=True)



In [ ]:
df['LargestPropertyUseType'].mode()

In [ ]:
# plots the distribution of LargestPropertyUseType
df.info()
#df['LargestPropertyUseType'].mode() 
df[['LargestPropertyUseType']].value_counts()[:10].plot(kind='barh')

In [ ]:
df.hist(column='ENERGYSTARScore', bins=20)

In [ ]:
df.dtypes

In [ ]:
#After Cleaning
#fig, ax = plt.subplots(figsize=(20,20))         # Sample figsize in inches
#sns.heatmap(df.corr(), annot=True , ax = ax)     # annot : shows value too

fig, ax = plt.subplots(figsize=(25,20))         # Sample figsize in inches
sns.heatmap(df.corr(), annot=True , ax = ax , cmap = 'RdYlGn', center=0)     # annot : shows value too

In [ ]:
# groupby analysis 
display(df['BuildingType'].value_counts())
df_BuildingType = df.groupby('BuildingType', axis=0).sum()

# note: the output of the groupby method is a `groupby' object. 
# we can not use it further until we apply a function (eg .sum())
print(type(df.groupby('BuildingType', axis=0)))

from IPython.display import display
    
df_BuildingType.head(10)


In [ ]:
df_BuildingType['TotalGHGEmissions'].plot(kind='pie',
                            figsize=(10, 10),
                            autopct='%1.1f%%', # add in percentages
                            startangle=45,     # start angle 90°
                            shadow=True,       # add shadow      
                            )
plt.title('TotalGHGEmissions distribution by BuidingType', y=1) 

# Hece we can say that NonResidential Type of Building are contributing highest to the Green HOuse Gas Emission

In [ ]:

df_BuildingType['NaturalGas(kBtu)'].plot(kind='pie',
                            figsize=(8, 8),
                            autopct='%1.1f%%', # add in percentages
                            startangle=45,     # start angle 90°
                            shadow=True,       # add shadow  
                                           
                           # labels=None,         # turn off labels on pie chart
                           # pctdistance=1.12,    # the ratio between the center of each pie slice and the start of the text generated by autopct 
                            )
plt.title('NaturalGas(kBtu) distribution by BuidingType', y=1) 


In [ ]:
world_map = folium.Map()

# display world map
world_map

In [ ]:
# lat and long of seattle
latitude = 47.6062
longitude = -122.3321

# 47.6062, and the longitude is -122.3321   

In [ ]:
# create map and display it
seattle_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of Seattle
seattle_map

In [ ]:
# instantiate a feature group 
buildings = folium.map.FeatureGroup()

# BuildingID whose data we have
for lat, lng, in zip(df.Latitude, df.Longitude):
    buildings.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=3, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add buildings to map
seattle_map.add_child(buildings)
# More data available from .

## Creating train and test dataset

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
test.head(5)